In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn import (datasets, naive_bayes, neighbors, neural_network, model_selection)

### Importing correct data split of 40,000

In [3]:
filename = "C:\\Users\\jakes\\S_train.txt"
S_in = np.loadtxt(filename, delimiter=',')
temp = S_in

filename = "C:\\Users\\jakes\\S_trainY.txt"
S_inY = np.loadtxt(filename, delimiter=',')
tempY = S_inY

filename = "C:\\Users\\jakes\\S_val.txt"
S_out = np.loadtxt(filename, delimiter=',')
tempO = S_out

filename = "C:\\Users\\jakes\\S_valY.txt"
S_outY = np.loadtxt(filename, delimiter=',')
tempOY = S_outY

In [4]:
[N,dim]=np.shape(S_in)

S_in = temp
S_inY = tempY

S_out = tempO
S_outY = tempOY

for i in range(dim):
    mean = np.mean(S_in[:,i])
    S_in[:,i] = S_in[:,i] - mean
    
    std = np.std(S_in[:,i])
    S_in[:,i] = S_in[:,i] / std

    S_out[:,i] = S_out[:,i] - mean
    S_out[:,i] = S_out[:,i] / std
    
    
#Reshaping data

S_inR = S_in
S_outR = S_out

S_inR = np.reshape(S_in, (len(S_in), 32,32,3))
S_outR = np.reshape(S_out, (len(S_out), 32,32,3))

### Finalized CNN

In [5]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), kernel_regularizer=tf.keras.regularizers.L1(l1=1e-7)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.L1(l1=1e-7)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.L1(l1=1e-7)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L1(l1=1e-7)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L1(l1=1e-7)),
    tf.keras.layers.Dense(10),
])

model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model2.fit(S_inR, S_inY, epochs=20,
                   validation_data=(S_outR, S_outY))

Epoch 1/20
1250/1250 [==============================] - 43s 31ms/step - loss: 1.4592 - accuracy: 0.4710 - val_loss: 1.1639 - val_accuracy: 0.5902
Epoch 2/20
1250/1250 [==============================] - 37s 30ms/step - loss: 1.0509 - accuracy: 0.6328 - val_loss: 0.9922 - val_accuracy: 0.6504
Epoch 3/20
1250/1250 [==============================] - 37s 30ms/step - loss: 0.8804 - accuracy: 0.6933 - val_loss: 0.8903 - val_accuracy: 0.6911
Epoch 4/20
1250/1250 [==============================] - 36s 29ms/step - loss: 0.7711 - accuracy: 0.7300 - val_loss: 0.8616 - val_accuracy: 0.7037
Epoch 5/20
1250/1250 [==============================] - 36s 29ms/step - loss: 0.6778 - accuracy: 0.7653 - val_loss: 0.8251 - val_accuracy: 0.7134
Epoch 6/20
1250/1250 [==============================] - 36s 29ms/step - loss: 0.6008 - accuracy: 0.7904 - val_loss: 0.8552 - val_accuracy: 0.7140
Epoch 7/20
1250/1250 [==============================] - 37s 29ms/step - loss: 0.5321 - accuracy: 0.8147 - val_loss: 0.8576 -

In [11]:
model2acc = history.history['accuracy']
model2valAcc = history.history['val_accuracy']
model2valLoss = history.history['val_loss']

In [7]:
m2pred = model2.predict(S_outR)

err = tf.keras.losses.SparseCategoricalCrossentropy()

m2Eout = err(S_outY, m2pred)

print(m2Eout)

313/313 [==============================] - 8s 10ms/step
tf.Tensor(1.9353856, shape=(), dtype=float32)


In [12]:
m2Acc = (S_outY==m2pred.argmax(axis=1)).sum()/S_outY.shape[0]
print(m2Acc)

0.7112


In [14]:
np.mean(model2valAcc)

0.7048349946737289